# Title here

Description here

## Table of contents
- [1. Business undestanding](#business)
- [2. Data understanding](#data)
    - [2.1. Gathering data](#gather)
    - [2.2. Assessing data](#assess)
- [3. Prepare data](#prepare)
- [4. Data modeling](#model)
- [5. Evaluate the results](#eval)
- [6. Deploy](#deploy)

<a name="business"></a>
## 1. Business understanding

Text text

> Question 1 \
> Question 2 \
> Question 3 \
> Question 4

<a name="data"></a>
## 2. Data understanding

Text text

<a name="gather"></a>
   

<a name="gather"></a>
### 2.1. Gathering data

Text text

<a name="assess"></a>
### 2.2. Assessing data

Text text

<a name="prepare"></a>
## 3. Prepare data

Text text

<a name="model"></a>
## 4. Data modeling

Text text

<a name="eval"></a>
## 5. Evaluate the results

Text text

<a name="deploy"></a>
## 6. Deploy

Text text